In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import multifidelityfunctions as mff
import multiLevelCoSurrogates as mlcs
import more_itertools
import pyDOE

plot_dir = 'plots/'

np.set_printoptions(precision=5, linewidth=180)

# Functions

In [ ]:
def sphere(X):
    return np.sum(1 - X**2, axis=1)

In [ ]:
def forrester(X):
    term1 = (6*X - 2)**2
    term2 = np.sin(12*X - 4)

    return np.sum(22 - (term1 * term2 + 6.03), axis=1)

In [ ]:
def ackley(X):
    tau = 2*np.pi
    
    sum_squares = np.sum(X**2, axis=1)
    sum_cos = np.sum(np.cos(tau*X), axis=1)
    
    term1 = -20*np.exp(-0.2 * np.sqrt(0.5*sum_squares))
    term2 = -np.exp(0.5*sum_cos)
    return term1 + term2 + np.e + 20

In [ ]:
# @mff.row_vectorize
# def himmelblau(xx):
#     pass

himmelblau = mff.himmelblau_hf

In [ ]:
def rastrigin(X):
    N = X.shape[1]
    A = 10
    tau = 2*np.pi
    
    term1 = A * N
    term2a = X**2
    term2b = A * np.cos(tau*X)
    term2 = np.sum(term2a - term2b, axis=1)
    return term1 + term2

# Historgram of test-sample distribution

In [ ]:
xrange = np.linspace(-1,1,201).reshape(-1,1)

plt.plot(xrange, sphere(xrange), label='function')
ax1 = plt.gca()
ax1.set_xlabel('x')
ax1.set_ylabel('y')

ax2 = ax1.twinx()
sample = mlcs.sample_by_function(sphere, ndim=1, n_samples=1000, 
                                 minimize=True, range_in=(-1,1), range_out=(0,1))
ax2.hist(sample, bins=80, color='C1', alpha=.5, label='minimize')
sample = mlcs.sample_by_function(sphere, ndim=1, n_samples=1000, 
                                 minimize=False, range_in=(-1,1), range_out=(0,1))
ax2.hist(sample, bins=80, color='C2', alpha=.5, label='maximize')
ax2.set_ylabel('Count')

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

plt.show()

# Error spread visualization

In [ ]:
xrange = np.linspace(0,1,101).reshape(-1,1)
plt.plot(xrange, forrester(xrange))
plt.ylim([0,22.5])
plt.show()

In [ ]:
y = forrester(np.linspace(0,1,10001).reshape(-1,1))
print('Output range should be [0, n]:')
print(f'[{np.min(y)}, {np.max(y)}]')

In [ ]:
archive = mlcs.CandidateArchive(ndim=1)#, fidelities=['high', 'low', 'high-low'])

n_samples = 5

np.random.seed(20160501)
x = np.random.rand(4, 1)
print(x)
# x = np.linspace(0,1,n_samples).reshape(-1,1)

y = forrester(x)
archive.addcandidates(x, forrester(x))#, fidelity='high')

surr = mlcs.Surrogate.fromname('Kriging', archive)#, kernel='Matern')
surr.retrain()

plt.plot(xrange, forrester(xrange))
plt.plot(xrange, surr.predict(xrange.reshape(-1,1)))
plt.scatter(*archive.getcandidates())
plt.ylim([0,22.5])
plt.show()

In [ ]:
def test_spacing_of_sample(test_sample, test_func):
    sample_size = test_sample.shape[0]
    sample = np.sort(test_sample, axis=0)
    
    x = surr.predict(sample).reshape(-1, 1)
    y = test_func(sample).reshape(-1, 1)
    square_errors = (x - y)**2
    print(f'MSE: {np.mean(square_errors)}')

    edges = (test_sample[:-1] + test_sample[1:]) / 2
    edges = [0] + edges.flatten().tolist() + [1]
    
    lengths = np.diff(edges).reshape(-1,1)

    return lengths, square_errors / lengths

In [ ]:
def add_error_spread_bars(ax, test_sample, test_func):
    test_sample = np.sort(test_sample, axis=0)

    edges = (test_sample[:-1] + test_sample[1:]) / 2
    edges = np.array([0] + edges.flatten().tolist() + [1])
    bar_midpoints = (edges[:-1] + edges[1:]) / 2

    lengths, lne = test_spacing_of_sample(test_sample, test_func=test_func)

    ax2 = ax.twinx()
    bar = ax2.bar(bar_midpoints, lne.flatten(), width=lengths.flatten(), bottom=-7, 
                  color='C3', alpha=.3, label='Square error / area covered by sample')
    return bar

In [ ]:
n_samples = 200
test_func = forrester

In [ ]:
np.random.seed(20160501)

test_sample = mlcs.sample_by_function(test_func, ndim=1, n_samples=n_samples, 
                                      minimize=False, range_in=(0,1), range_out=(0,22))
print(test_sample.shape)
ax1 = plt.subplot(111)
ax1.plot(xrange, test_func(xrange))
ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)))
ax1.scatter(test_sample, test_func(test_sample))

add_error_spread_bars(ax1, test_sample, test_func)
    
ax1.set_title('function-based probability distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])

plt.show()

In [ ]:
np.random.seed(20160501)
test_sample = np.random.rand(100,1)
ax1 = plt.subplot(111)
ax1.plot(xrange, test_func(xrange))
ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)))
ax1.scatter(test_sample, test_func(test_sample))

add_error_spread_bars(ax1, test_sample, test_func)

ax1.set_title('uniform random probability distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])
plt.show()

In [ ]:
np.random.seed(20160501)
test_sample = pyDOE.lhs(1, 100)
ax1 = plt.subplot(111)
ax1.plot(xrange, test_func(xrange))
ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)))
ax1.scatter(test_sample, test_func(test_sample))

add_error_spread_bars(ax1, test_sample, test_func)

ax1.set_title('LHS distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])
plt.show()

In [ ]:
sample_size = 1000

plt.figure(figsize=(18,5))

###########

np.random.seed(20160501)
test_sample = np.random.rand(100,1)
ax1 = plt.subplot(131)
ln1 = ax1.plot(xrange, test_func(xrange), label='True function')
ln2 = ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)), label='GP model')
sc1 = ax1.scatter(test_sample, test_func(test_sample), s=12, label=f'test sample (n={sample_size})')

br1 = add_error_spread_bars(ax1, test_sample, test_func)

ax1.set_title('Uniform random probability distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc=0)

###############

np.random.seed(20160501)
test_sample = pyDOE.lhs(1, 100)
ax1 = plt.subplot(132)
ax1.plot(xrange, test_func(xrange))
ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)))
ax1.scatter(test_sample, test_func(test_sample), s=12)

add_error_spread_bars(ax1, test_sample, test_func)

ax1.set_title('LHS distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])

##################

np.random.seed(20160501)
test_sample = mlcs.sample_by_function(test_func, ndim=1, n_samples=n_samples, 
                                      minimize=False, range_in=(0,1), range_out=(0,22))
ax1 = plt.subplot(133)
ax1.plot(xrange, test_func(xrange))
ax1.plot(xrange, surr.predict(xrange.reshape(-1,1)))
ax1.scatter(test_sample, test_func(test_sample), s=12)

add_error_spread_bars(ax1, test_sample, test_func)

ax1.set_title('Function-based probability distribution')
ax1.set_ylim([0,22.5])
ax1.set_xlim([0,1])

###################

plt.tight_layout()
plt.savefig(f'{plot_dir}normed_errors_per_section.png')
plt.savefig(f'{plot_dir}normed_errors_per_section.pdf')
plt.show()

In [ ]:
np.random.seed(20160501)

ax1 = plt.subplot(111)
y = test_func(xrange)
y_surr = surr.predict(xrange.reshape(-1,1))
abs_err = np.abs(y-y_surr)

ax1.plot(xrange, y, label='True function')
ax1.plot(xrange, y_surr, label='GP model')
ax1.plot(xrange, abs_err, label='Abs. error')
ax1.plot(xrange, abs_err * (y/np.max(y)), label='f(x)-weighted abs. error')

ax1.set_ylim([0,np.max(y)*1.05])
ax1.set_xlim([0,1])
ax1.legend(loc=2)
plt.show()

# Some simple examples

A number of cases where a model is trained on just 4 points in the 1D function case.
Intended for illustration of which models we want to consider as better/worse and how this is achieved using the FSS method

In [ ]:
def show_1d_examples(test_func, resolution, *, range_in=mlcs.ValueRange(0,1), title=''):
    xrange = np.linspace(*range_in,resolution+1).reshape(-1, 1)
    y = test_func(xrange)
    samples = np.array([
        [
            [0.0, 0.4, 0.6, 1.0],
#             [0.0, 0.33, 0.66, 1.0],
            [0.0, 0.6, 0.8, 1.0],
        ],
        [
#             [0.1, 0.3, 0.5, 0.7],
             [0.3, 0.4, 0.7, 0.9],
             [0.0, 0.1, 0.55, 0.9],
        ],
#         [
#             [0.1, 0.2, 0.3, 0.4],
#             [0.3, 0.4, 0.6, 0.7],
#             [0.6, 0.7, 0.8, 0.9],
#         ],
    ])
    
    shape = samples.shape
    fig, axes = plt.subplots(nrows=shape[0], ncols=shape[1], figsize=(4*shape[1], 4*shape[0]))
    add_legend = True

    for sample, ax in zip(samples.reshape(-1,4), axes.flatten()):
        sample = mlcs.rescale(sample, range_in=(0,1), range_out=range_in)
        x = sample.reshape(-1, 1)
        archive = mlcs.CandidateArchive(ndim=1)#, fidelities=['high', 'low', 'high-low'])
        archive.addcandidates(x, test_func(x))#, fidelity='high')

        surr = mlcs.Surrogate.fromname('Kriging', archive, kernel='Matern')
        surr.retrain()

        y_surr = surr.predict(xrange.reshape(-1,1))
        abs_err = np.abs(y-y_surr)
        scale = (y/np.max(y))*.98 + .01
        
        ax.plot(xrange, y, label='True function')
        ax.plot(xrange, y_surr, label='GP model')
        ax.plot(xrange, abs_err, label='Abs. error')
        ax.plot(xrange, abs_err * scale, label='f(x)-weighted abs. error')
        ax.scatter(*archive.getcandidates(), zorder=3, color='black', label='Training set')
        ax.set_ylim([-0.01,np.max(y)*1.05])
        ax.set_xlim([*range_in])

        if add_legend:
            ax.legend(loc=2)
            add_legend = False

    plt.suptitle(title)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
#     plt.show()

In [ ]:
name = 'Sphere'

show_1d_examples(sphere, 100, range_in=(-1,1), title=name)
plt.savefig(f'{plot_dir}examples_{name.lower()}.pdf')
plt.savefig(f'{plot_dir}examples_{name.lower()}.png')
plt.show()

In [ ]:
name = 'Forrester'

show_1d_examples(forrester, 100, title=name)
plt.savefig(f'{plot_dir}examples_{name.lower()}.pdf')
plt.savefig(f'{plot_dir}examples_{name.lower()}.png')
plt.show()

In [ ]:
name = 'Ackley'

show_1d_examples(ackley, 500, range_in=(-5, 5), title=name)
plt.savefig(f'{plot_dir}examples_{name.lower()}.pdf')
plt.savefig(f'{plot_dir}examples_{name.lower()}.png')
plt.show()

In [ ]:
name = 'Rastrigin'

show_1d_examples(rastrigin, 512, range_in=(-5.12, 5.12), title=name)
plt.savefig(f'{plot_dir}examples_{name.lower()}.pdf')
plt.savefig(f'{plot_dir}examples_{name.lower()}.png')
plt.show()

# mexican hat function

In [ ]:
x = np.linspace(-1, 1, 1001).reshape(-1,1)
a = 4
y = ((1 - (a*x)**2) * np.exp(-(a*x)**2 / 2) + .5) / 1.5

y_surr = y+.4

print('Output range should be [0, n]:')
print(f'[{np.min(y)}, {np.max(y)}]')

abs_err = np.abs(y-y_surr)
scale = (y/np.max(y))*.98 + .01

plt.plot(x, y, label='y = f(x)')
plt.plot(x, y_surr, label='y\' = g(x) = f(x) + 0.4')

plt.plot(x, abs_err, '--', label='|y-y\'|')
plt.plot(x, abs_err * (y/np.max(y)), '--', label='|y-y\'| / max(y)')

# plt.title('Fuction-weighted error influence')

plt.xlabel('x')
plt.xlim([-1,1])
plt.ylabel('y')
plt.ylim([0, 1.5])

plt.legend(loc=0)
plt.tight_layout()

plt.show()

By definition, the absolute error is exactly the given offset. The f(x)-weighted line however only matches that 'importance' at the optimum, while elsewhere dropping (much) lower to indicate that the errors in this area are much less interesting/relevant

In [ ]:
x = np.linspace(-1, 1, 1001).reshape(-1,1)
a = 4
y = ((1 - (a*x)**2) * np.exp(-(a*x)**2 / 2) + .5) / 1.5

y_surr = 1-y

print('Output range should be [0, n]:')
print(f'[{np.min(y)}, {np.max(y)}]')

abs_err = np.abs(y-y_surr)
scale = (y/np.max(y))*.98 + .01

plt.plot(x, y, label='y = f(x)')
plt.plot(x, y_surr, label='y\' = g(x) = 1 - f(x)')

plt.plot(x, abs_err, '--', label='|y-y\'|')
plt.plot(x, abs_err * (y/np.max(y)), '--', label='|y-y\'| / max(y)')

# plt.title('Fuction-weighted error influence')

plt.xlabel('x')
plt.xlim([-1,1])
plt.ylabel('y')
plt.ylim([0, 1.1])

plt.legend(loc=0)
plt.tight_layout()

plt.show()